In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import utils
import seaborn as sns
import importlib
from tqdm import tqdm
from joblib import Parallel, delayed
import multiprocessing as mp
import warping
import matplotlib.cm as cm
import custom_KM
import custom_damping_KM

path_figs = "./Figs/"

path = "./SL3/SL3"

path_misc = "./SL3_miscs/"

fingers = [0, 1, 2, 3, 4]

iti = 500

total_sub_num = 16

n_samples = 300
n_samples_left_right = 30


In [ ]:

importlib.reload(utils)

In [ ]:
subjs = pd.read_csv(path_misc + "subjs.csv")

In [ ]:
# subjs = utils.remove_error_trials(subjs)

In [ ]:
rank_df = pd.DataFrame(index = subjs.index, dtype='int')
for subnum, data in subjs.groupby('SubNum'):
    data = data.sort_values(by = ['BN', 'TN'])
    data['N'] = data.groupby(['BN', 'TN']).ngroup() + 1
    rank_df.loc[data.index, 'N'] = data['N'].astype('int')
        

subjs['N'] = rank_df['N'].astype('int')

In [ ]:
subjs['ET'] = subjs['MT'] + subjs['IPI0']

In [ ]:
subjs['ET'].median() // 5

## Mov Files:

In [ ]:
aligned_cut_force = pd.read_csv(utils.path_misc+'aligned_cut_force.csv', sep = '\t')

In [ ]:

forces = [1,2,3,4,5]

num_sequences = 4
num_cores = mp.cpu_count()

all_error_trials = {}


num_trials = 1200

all_error_trials = {}

all_trials_MT = {}

seq_types = subjs['seqType'].unique()

for seqnum, seq_data in subjs.groupby('seqType'):
    for hand, hand_data in seq_data.groupby('board'):


        for subind, (subnum, subdata) in enumerate(hand_data.groupby('SubNum')):
            
            error_ranks = subdata[subdata['seqError'] > 0]['N'].unique().astype('int')
            all_error_trials[seqnum, hand, subnum] = error_ranks 




for seqnum, seq_data in subjs.groupby('seqType'):
    for hand, hand_data in seq_data.groupby('board'):
        for subind, (subnum, subdata) in enumerate(hand_data.groupby('SubNum')):
            MT = subdata['MT'].to_numpy()
            all_trials_MT[seqnum, hand, subnum] = MT


In [ ]:
from sklearn.decomposition import PCA
forces = [1,2,3,4,5]
import pickle

fig, axs = plt.subplots(4*2, 2, figsize = (12, 40))

num_sequences = 4
num_cores = mp.cpu_count()

reduced_forces_correct_dict = {}
reduced_forces_all_dict = {}
components = {}

n_dim_reduction = 15


# num_trials = 1200


seq_types = subjs['seqType'].unique()

for seqnum, seq_data in aligned_cut_force.groupby('seqType'):
    for hand, hand_data in seq_data.groupby('board'):
        for subind, (subnum, subdata) in enumerate(hand_data.groupby('SubNum')):
            subdata_correct = utils.remove_error_trials(subdata)
            pivot_data = subdata_correct.pivot_table(index = ['N'], columns = ['Force_Number', 'time'], values = 'Force_Value').dropna().to_numpy()
            pca = PCA()
            pca.fit(pivot_data)
            components[seqnum, hand, subnum] = pca.components_[:n_dim_reduction]

            explained_variance = pca.explained_variance_ratio_
            dim_reduced = np.argmax(np.cumsum(explained_variance) >= 0.9)
            # print(dim_reduced)

            reduced_forces_correct_dict[seqnum, hand, subnum] = pca.transform(pivot_data)[:,:n_dim_reduction]
            pivot_data = subdata.pivot_table(index = ['N'], columns = ['Force_Number', 'time'], values = 'Force_Value').dropna().to_numpy()
            reduced_forces_all_dict[seqnum, hand, subnum] = pca.transform(pivot_data)[:,:n_dim_reduction]


            seq_ind = seqnum - 1
            ax_ind = (seq_ind * 2 + hand, subind)
            axs[ax_ind].plot(np.arange(1, len(explained_variance) + 1), explained_variance, marker = 'o', linestyle = '--')
            # axs[ax_ind].set_yscale('log')
            axs[ax_ind].set_title(f'Seq {seqnum} hand {hand} sub {subnum}')
            axs[ax_ind].set_xlabel('Principal Component')
            axs[ax_ind].set_ylabel('Explained Variance Ratio')
            # axs[ax_ind].set_ylim([1e-3, 1])

            

In [ ]:

# forces = [1,2,3,4,5]
# import pickle


# distances_dict_reduced = {}

# num_sequences = 4
# num_cores = mp.cpu_count()




# # num_trials = 1200

# seq_types = subjs['seqType'].unique()

# for idx, ((seqnum, hand, subnum), force) in enumerate(reduced_forces_all_dict.items()):
#     error_trials = all_error_trials[seqnum, hand, subnum]
#     correct_trials = np.setdiff1d(np.arange(0, force.shape[0]), error_trials - 1)
#     distances = pd.DataFrame(index = correct_trials, columns = correct_trials,
#                             dtype='float')

#     for col1idx, col1 in enumerate(tqdm(correct_trials, desc = f"Processing sub {subnum}seq {seqnum} hand {hand}", leave=False)):
#         for col2idx, col2 in enumerate(correct_trials):
#             if col2idx >= col1idx:
#                 distance = np.linalg.norm(force[col1] - force[col2])
#                 distances.loc[col1, col2] = distance
#                 distances.loc[col2, col1] = distance

#     distances = distances.dropna(how = 'all').dropna(axis = 1)
#     distances_dict_reduced[seqnum, hand, subnum] = distances
    
#     # sns.heatmap(distances, cmap = 'coolwarm', ax = axs[idx // 2, idx % 2])

#     # drwaing lines at trial numbers 400 and 800
#     # for trial in [400, 800]:
#     #     axs[idx//2, idx % 2].annotate('', xy = (0, trial), xytext = (-1, trial), arrowprops = dict(facecolor = 'green', shrink = 0.05, headwidth = 0.1))





In [ ]:
# with open(utils.path_misc + 'distances_dict_reduced.pkl', 'wb') as f:
#     pickle.dump(distances_dict_reduced, f)


In [ ]:
import pickle
with open(utils.path_misc + 'distances_dict_reduced.pkl', 'rb') as f:
    distances_dict_reduced = pickle.load(f)

In [ ]:
# %matplotlib notebook

In [ ]:
%matplotlib widget

In [ ]:
import importlib
importlib.reload(custom_damping_KM)

In [ ]:
#project and plot random walk on the first 3 principal components interactively in 3D
import plotly.graph_objects as go
from pykalman import KalmanFilter
import numpy.ma as ma

reduced_forces_all_filtered_diag = {}

for idx, ((seqnum, hand, subnum), forces) in enumerate(reduced_forces_all_dict.items()):
    if subnum != 8:
        kf = custom_damping_KM.KalmanFilter(n_dim_state = n_dim_reduction, n_dim_obs = n_dim_reduction, observation_matrices = np.eye(n_dim_reduction), 
                    transition_offsets = np.zeros(n_dim_reduction), observation_offsets = np.zeros(n_dim_reduction), initial_state_mean=forces[0], initial_state_covariance=np.zeros((n_dim_reduction, n_dim_reduction)),
                    em_vars=['transition_covariance', 'observation_covariance', 'transition_matrices'])
        

        error_inds = all_error_trials[(seqnum, hand, subnum)]
        mask = np.zeros_like(forces)
        mask[error_inds-1, :] = 1
        forces = ma.masked_array(forces, mask)
        kf = kf.em(forces, n_iter=5)
        filtered_states, _ = kf.filter(forces)
        reduced_forces_all_filtered_diag[seqnum, hand, subnum] = filtered_states

        #change color of lines continuously with index
        color = np.arange(len(forces))
        fig = go.Figure(data=[go.Scatter3d(x=filtered_states[:,0], y=filtered_states[:,1], z=filtered_states[:,2], mode='lines', line=dict(color=color, colorscale='Viridis', width=1, showscale = True))])
        #show color progression in colorbar
        # fig.update_layout(coloraxis_colorbar=dict(title='Index', tickvals=[0, len(forces)], ticktext=['Start', 'End']))

        # fig = go.Figure(data=[go.Scatter3d(x=forces[:,0], y=forces[:,1], z=forces[:,2], mode='lines')])
        fig.update_layout(scene = dict(
                            xaxis_title='PC1',
                            yaxis_title='PC2',
                            zaxis_title='PC3'), 
                            width=800,
                            height=800, 
                            margin=dict(l=100, r=100, b = 100, t=100),
                            title_text = f'Seq {seqnum} hand {hand} sub {subnum}')
                            
        fig.show()
        print(kf.loglikelihood(forces))
        print(np.diag(kf.transition_matrices))


In [ ]:
np.diag(kf.transition_matrices)

In [ ]:
kf.loglikelihood(forces)

In [ ]:
#project and plot random walk on the first 3 principal components interactively in 3D
import plotly.graph_objects as go
from pykalman import KalmanFilter
import numpy.ma as ma


reduced_forces_all_filtered_diag = {}

for idx, ((seqnum, hand, subnum), forces) in enumerate(reduced_forces_all_dict.items()):
    if subnum != 8:
        kf = custom_KM.KalmanFilter(n_dim_state = n_dim_reduction, n_dim_obs = n_dim_reduction, transition_matrices=np.eye(n_dim_reduction), observation_matrices = np.eye(n_dim_reduction), 
                    transition_offsets = np.zeros(n_dim_reduction), observation_offsets = np.zeros(n_dim_reduction), initial_state_mean=forces[0], initial_state_covariance=np.zeros((n_dim_reduction, n_dim_reduction)),
                    em_vars=['transition_covariance', 'observation_covariance'])
        

        error_inds = all_error_trials[(seqnum, hand, subnum)]
        mask = np.zeros_like(forces)
        mask[error_inds-1, :] = 1
        forces = ma.masked_array(forces, mask)
        kf = kf.em(forces, n_iter=5)
        filtered_states, _ = kf.filter(forces)
        reduced_forces_all_filtered_diag[seqnum, hand, subnum] = filtered_states

        #change color of lines continuously with index
        color = np.arange(len(forces))
        fig = go.Figure(data=[go.Scatter3d(x=filtered_states[:,0], y=filtered_states[:,1], z=filtered_states[:,2], mode='lines', line=dict(color=color, colorscale='Viridis', width=1, showscale = True))])
        #show color progression in colorbar
        # fig.update_layout(coloraxis_colorbar=dict(title='Index', tickvals=[0, len(forces)], ticktext=['Start', 'End']))

        # fig = go.Figure(data=[go.Scatter3d(x=forces[:,0], y=forces[:,1], z=forces[:,2], mode='lines')])
        fig.update_layout(scene = dict(
                            xaxis_title='PC1',
                            yaxis_title='PC2',
                            zaxis_title='PC3'), 
                            width=800,
                            height=800, 
                            margin=dict(l=100, r=100, b = 100, t=100),
                            title_text = f'Seq {seqnum} hand {hand} sub {subnum}')
                            
        fig.show()
        print(kf.loglikelihood(forces))




In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D
# import seaborn as sns

# for idx, ((seqnum, hand, subnum), distances) in enumerate(distances_dict_reduced.items()):
#     if idx != 0:
#         continue
    
#     sub_data = subjs[(subjs['seqType'] == seqnum) & (subjs['board'] == hand) & (subjs['SubNum'] == subnum)].copy().sort_values(by=['N']).reset_index()
    
#     next_BN = sub_data['BN'].shift(-1)
#     sub_data['next_BN'] = next_BN
#     sub_data['next_N_in_next_BN'] = (sub_data['BN'] + 1 == sub_data['next_BN'])

#     block_change_trials_idx = sub_data[sub_data['next_N_in_next_BN']].index
#     block_change_trials_on_plot = [np.argmin(np.abs(distances.index - trial)) for trial in block_change_trials_idx]

#     first_day_change_trial_on_plot = np.argmin(np.abs(distances.index - 399))
#     second_day_change_trial_on_plot = np.argmin(np.abs(distances.index - 799))

#     # Prepare X, Y, and Z data for 3D surface plot
#     x = np.arange(distances.shape[0])
#     y = np.arange(distances.shape[1])
#     X, Y = np.meshgrid(x, y)
    
#     # For Z, use distances, and for Z heights, use ET values repeated across Y axis
#     Z = distances.values  # distances as color intensity for heatmap effect

#     error_trials = all_error_trials[seqnum, hand, subnum]
#     correct_trials = np.setdiff1d(np.arange(0, sub_data.shape[0]), error_trials - 1)
#     correct_data = sub_data.iloc[correct_trials]
#     ET_values = correct_data['ET'].values

#     Z_et = np.outer(ET_values, np.ones(distances.shape[1]))  # Repeat ET values across Y axis
    
#     # Initialize the 3D plot
#     fig = plt.figure(figsize=(10, 12))
#     ax = fig.add_subplot(111, projection='3d')

#     # Create a 3D surface plot
#     surf = ax.plot_surface(X, Y, Z_et, facecolors=plt.cm.coolwarm(Z), rstride=1, cstride=1, linewidth=0, antialiased=False)

#     # Add lines for day changes
#     ax.axvline(x=first_day_change_trial_on_plot, color='black', linestyle='--')
#     ax.axvline(x=second_day_change_trial_on_plot, color='black', linestyle='--')
#     ax.axhline(y=first_day_change_trial_on_plot, color='black', linestyle='--')
#     ax.axhline(y=second_day_change_trial_on_plot, color='black', linestyle='--')

#     # Customize ticks for block change trials
#     ax.set_xticks(block_change_trials_on_plot)
#     ax.set_xticklabels(block_change_trials_idx, rotation=45)
#     ax.set_yticks(block_change_trials_on_plot)
#     ax.set_yticklabels(block_change_trials_idx)

#     # Set labels and title for 3D plot
#     ax.set_xlabel("Trials (X-axis)")
#     ax.set_ylabel("Trials (Y-axis)")
#     ax.set_zlabel("ET (Z-axis)")
#     ax.set_title(f"3D Heatmap of Distances with ET as Third Axis (Seq {seqnum}, Hand {hand}, Sub {subnum})")

#     # Color bar for heatmap effect based on distances
#     m = plt.cm.ScalarMappable(cmap="coolwarm")
#     m.set_array(Z)
#     fig.colorbar(m, ax=ax, shrink=0.5, aspect=5)

#     plt.show()


In [ ]:
import numpy as np
import plotly.graph_objects as go
from scipy.ndimage import gaussian_filter

# Assuming distances_dict_reduced, subjs, and all_error_trials are already defined
for idx, ((seqnum, hand, subnum), distances) in enumerate(distances_dict_reduced.items()):
    if idx != 1:
        continue  # Only run for the first entry
    
    sub_data = subjs[(subjs['seqType'] == seqnum) & (subjs['board'] == hand) & (subjs['SubNum'] == subnum)].copy().sort_values(by=['N']).reset_index()
    
    next_BN = sub_data['BN'].shift(-1)
    sub_data['next_BN'] = next_BN
    sub_data['next_N_in_next_BN'] = (sub_data['BN'] + 1 == sub_data['next_BN'])

    block_change_trials_idx = sub_data[sub_data['next_N_in_next_BN']].index
    block_change_trials_on_plot = [np.argmin(np.abs(distances.index - trial)) for trial in block_change_trials_idx]

    first_day_change_trial_on_plot = np.argmin(np.abs(distances.index - 399))
    second_day_change_trial_on_plot = np.argmin(np.abs(distances.index - 799))

    # Prepare X, Y, and Z data for 3D surface plot
    x = np.arange(distances.shape[0])
    y = np.arange(distances.shape[1])
    X, Y = np.meshgrid(x, y)
    
    # Set up Z for color and Z_et for height
    Z = distances.values  # distances as color intensity for heatmap effect

    # Average Smoothing Z values for better visualization
    # Z = (Z + np.roll(Z, 1, axis=0) + np.roll(Z, -1, axis=0) + np.roll(Z, 1, axis=1) + np.roll(Z, -1, axis=1)) / 5
    # Z = gaussian_filter(Z, sigma=4)

    

    error_trials = all_error_trials[seqnum, hand, subnum]
    correct_trials = np.setdiff1d(np.arange(0, sub_data.shape[0]), error_trials - 1)
    correct_data = sub_data.iloc[correct_trials]
    ET_values = correct_data['ET'].values

    Z_et = np.outer(ET_values, np.ones(distances.shape[1]))  # Repeat ET values across Y axis
    
    # Convert the 'coolwarm' colormap from matplotlib to Plotly colorscale
    cmap = cm.get_cmap('coolwarm')
    coolwarm_colorscale = [[i / 255, f'rgb({int(r*255)}, {int(g*255)}, {int(b*255)})'] for i, (r, g, b, _) in enumerate(cmap(np.linspace(0, 1, 256)))]

    #restricting colorbar
    Z_et = np.clip(Z_et, 1000, 3000)



    # Create Plotly surface plot
    fig = go.Figure(data=[go.Surface(z=Z, x=X, y=Y, surfacecolor=Z_et, colorscale=coolwarm_colorscale, colorbar=dict(title="Z Value"))])

    # Add vertical lines for day changes as annotations
    # fig.add_trace(go.Scatter3d(x=[first_day_change_trial_on_plot, second_day_change_trial_on_plot],
    #                            y=[0, distances.shape[1]],
    #                            z=[Z_et.min(), Z_et.max()],
    #                            mode='lines',
    #                            line=dict(color='black', dash='dash')))

    # Customize layout
    fig.update_layout(title=f"3D Heatmap of Distances with ET as Third Axis (Seq {seqnum}, Hand {hand}, Sub {subnum})",
                      scene=dict(
                          xaxis_title="Trials (X-axis)",
                          yaxis_title="Trials (Y-axis)",
                          zaxis_title="ET (Z-axis)",
                          xaxis=dict(tickvals=block_change_trials_on_plot, ticktext=block_change_trials_idx),
                          yaxis=dict(tickvals=block_change_trials_on_plot, ticktext=block_change_trials_idx)),
                      width=1200, height=1200, margin=dict(l=65, r=50, b=65, t=90))

    fig.show()


In [ ]:
%matplotlib inline

In [ ]:
for idx, ((seqnum, hand, subnum), distances) in enumerate(distances_dict_reduced.items()):
    # if idx != 0:
    #     continue

    if subnum == 8:
        continue
    
    sub_data = subjs[(subjs['seqType'] == seqnum) & (subjs['board'] == hand) & (subjs['SubNum'] == subnum)].copy().sort_values(by = ['N']).reset_index()

    

    next_BN = sub_data['BN'].shift(-1)
    sub_data['next_BN'] = next_BN
    sub_data['next_N_in_next_BN'] = (sub_data['BN'] + 1 == sub_data['next_BN'])

    block_change_trials_idx = sub_data[sub_data['next_N_in_next_BN']].index
    block_change_trials_on_plot = [np.argmin(np.abs(distances.index - trial)) for trial in block_change_trials_idx]

    first_day_change_trial_on_plot = np.argmin(np.abs(distances.index - 399))
    second_day_change_trial_on_plot = np.argmin(np.abs(distances.index - 799))





    # Start plotting
    fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(10, 12), sharex=True, gridspec_kw={'height_ratios': [4, 1, 1, 1]})



    # Plot heatmap
    sns.heatmap(distances, cmap='coolwarm', ax=ax1, cbar=False)
    ax1.axvline(first_day_change_trial_on_plot, color='black', linestyle='--')
    ax1.axvline(second_day_change_trial_on_plot, color='black', linestyle='--')
    ax1.axhline(first_day_change_trial_on_plot, color='black', linestyle='--')
    ax1.axhline(second_day_change_trial_on_plot, color='black', linestyle='--')

    # Set x-ticks and y-ticks with block change trials for the heatmap
    ax1.set_xticks(block_change_trials_on_plot)
    ax1.set_xticklabels(block_change_trials_idx)
    ax1.set_yticks(block_change_trials_on_plot)
    ax1.set_yticklabels(block_change_trials_idx)

    # Plot ET line plot below the heatmap for correct trials
    error_trials = all_error_trials[seqnum, hand, subnum]
    correct_trials = np.setdiff1d(np.arange(num_trials), error_trials - 1)
    correct_data = sub_data.iloc[correct_trials]
    ax2.scatter(np.arange(len(correct_data['ET'])) , correct_data['ET'].values, label='ET', s = 0.5)
    ax2.axvline(first_day_change_trial_on_plot, color='black', linestyle='--')
    ax2.axvline(second_day_change_trial_on_plot, color='black', linestyle='--')

    # Align x-ticks for both subplots
    ax2.set_xticks(block_change_trials_on_plot)
    ax2.set_xticklabels(block_change_trials_idx, rotation=45)
    ax2.set_ylabel('ET')

    #despine ax2
    sns.despine(ax=ax2, left=True, bottom=True)


    # Plot delta Z line plot below the heatmap for correct trials
    filtered_states = reduced_forces_all_filtered_diag[seqnum, hand, subnum]
    filtered_states_correct = filtered_states[correct_trials]
    deltaz = np.diff(filtered_states_correct, axis = 0)
    deltaz = np.linalg.norm(deltaz, axis = 1)
    ax3.scatter(np.arange(1, len(deltaz) + 1), deltaz, s = 0.5)
    ax3.axvline(first_day_change_trial_on_plot, color='black', linestyle='--')
    ax3.axvline(second_day_change_trial_on_plot, color='black', linestyle='--')
    ax3.set_ylabel('Delta Z')
    sns.despine(ax=ax3, left=True, bottom=True)



    # Plot innovation line plot below the heatmap for correct trials
    forces = reduced_forces_all_dict[seqnum, hand, subnum]
    forces_correct = forces[correct_trials]
    innovation = np.linalg.norm(forces_correct - filtered_states_correct, axis = 1)
    ax4.scatter(np.arange(0, len(innovation)), innovation, s = 0.5)
    ax4.axvline(first_day_change_trial_on_plot, color='black', linestyle='--')
    ax4.axvline(second_day_change_trial_on_plot, color='black', linestyle='--')
    ax4.set_ylabel('Innovation')
    sns.despine(ax=ax4, left=True, bottom=True)




    #setting title for whole figure
    fig.suptitle(f"Seq {seqnum} Hand {hand} Sub {subnum}")
    

    # Add labels and titles as needed
    # ax1.set_title("Heatmap with Day Change Lines")
    # ax2.set_title("ET over Trials")

    plt.tight_layout(rect=[0, 0, 1, 0.97])
    plt.subplots_adjust(hspace=0.05)  # Reduce space between subplots for alignment
    plt.show()

        
    

In [ ]:
# fig, axs = plt.subplots(4*2, 2, figsize = (12, 40))
from scipy.stats import linregress
for idx, ((seqnum, hand, subnum), distances) in enumerate(distances_dict_reduced.items()):
    # if idx != 0:
    #     continue

    if subnum == 8:
        continue
    
    sub_data = subjs[(subjs['seqType'] == seqnum) & (subjs['board'] == hand) & (subjs['SubNum'] == subnum)].copy().sort_values(by = ['N']).reset_index()

    # Plot ET line plot below the heatmap for correct trials
    error_trials = all_error_trials[seqnum, hand, subnum]
    correct_trials = np.setdiff1d(np.arange(num_trials), error_trials - 1)
    correct_data = sub_data.iloc[correct_trials]


    # Plot delta Z line plot below the heatmap for correct trials
    filtered_states = reduced_forces_all_filtered_diag[seqnum, hand, subnum]
    filtered_states_correct = filtered_states[correct_trials]
    deltaz = np.diff(filtered_states_correct, axis = 0)
    deltaz = np.linalg.norm(deltaz, axis = 1)


    # Plot innovation line plot below the heatmap for correct trials
    forces = reduced_forces_all_dict[seqnum, hand, subnum]
    forces_correct = forces[correct_trials]
    innovation = np.linalg.norm(forces_correct - filtered_states_correct, axis = 1)


    # Smooth the ET values using a rolling median
    window_size = 5  # Adjust as needed
    smoothed_ET = correct_data['ET']

    # Remove NaN values resulting from the rolling operation
    valid_indices = smoothed_ET.notna()
    smoothed_ET = smoothed_ET[valid_indices]
    innovation = innovation[valid_indices]  # Align innovation data


    # Perform linear regression
    slope, intercept, r_value, p_value, std_err = linregress(smoothed_ET, innovation)

    # Compute the best-fit line
    best_fit_line = slope * smoothed_ET + intercept

    # Compute coefficient of determination (R^2)
    r_squared = r_value**2

    print(f"Seq {seqnum} Hand {hand} Sub {subnum}", np.corrcoef(smoothed_ET, innovation)[0, 1])

    # Plot scatter plot and best-fit line
    fig = plt.figure(figsize=(6, 6))
    plt.scatter(smoothed_ET, innovation, s=0.5, label='Data points', alpha=0.7)
    plt.plot(smoothed_ET, best_fit_line, color='red', label=f'Best-fit line\n$R^2$={r_squared:.2f}, p={p_value:.2e}')
    plt.xlim(left = 0)
    plt.ylim(bottom = 0)


    plt.title(f"Seq {seqnum} Hand {hand} Sub {subnum}")

    plt.xlabel('ET')
    plt.ylabel('Innovation')
    plt.legend()

        
    

In [ ]:
# fig, axs = plt.subplots(4*2, 2, figsize = (12, 40))
from scipy.stats import linregress
for idx, ((seqnum, hand, subnum), distances) in enumerate(distances_dict_reduced.items()):
    # if idx != 0:
    #     continue

    if subnum == 8:
        continue
    
    sub_data = subjs[(subjs['seqType'] == seqnum) & (subjs['board'] == hand) & (subjs['SubNum'] == subnum)].copy().sort_values(by = ['N']).reset_index()

    # Plot ET line plot below the heatmap for correct trials
    error_trials = all_error_trials[seqnum, hand, subnum]
    correct_trials = np.setdiff1d(np.arange(num_trials), error_trials - 1)
    correct_data = sub_data.iloc[correct_trials]


    # Plot delta Z line plot below the heatmap for correct trials
    filtered_states = reduced_forces_all_filtered_diag[seqnum, hand, subnum]
    filtered_states_correct = filtered_states[correct_trials]
    deltaz = np.diff(filtered_states_correct, axis = 0)
    deltaz = np.linalg.norm(deltaz, axis = 1)


    # Plot innovation line plot below the heatmap for correct trials
    forces = reduced_forces_all_dict[seqnum, hand, subnum]
    forces_correct = forces[correct_trials]
    innovation = np.linalg.norm(forces_correct - filtered_states_correct, axis = 1)


    # Smooth the ET values using a rolling median
    window_size = 5  # Adjust as needed
    smoothed_ET = correct_data['ET']
    smoothed_ET = smoothed_ET[1:]


    # Perform linear regression
    slope, intercept, r_value, p_value, std_err = linregress(smoothed_ET, deltaz)

    # Compute the best-fit line
    best_fit_line = slope * smoothed_ET + intercept

    # Compute coefficient of determination (R^2)
    r_squared = r_value**2

    print(f"Seq {seqnum} Hand {hand} Sub {subnum}", np.corrcoef(smoothed_ET, deltaz)[0, 1])

    # Plot scatter plot and best-fit line
    fig = plt.figure(figsize=(6, 6))
    plt.scatter(smoothed_ET, deltaz, s=0.5, label='Data points', alpha=0.7)
    plt.plot(smoothed_ET, best_fit_line, color='red', label=f'Best-fit line\n$R^2$={r_squared:.2f}, p={p_value:.2e}')
    plt.xlim(left = 0)
    plt.ylim(bottom = 0)


    plt.title(f"Seq {seqnum} Hand {hand} Sub {subnum}")

    plt.xlabel('ET')
    plt.ylabel('delta z')
    plt.legend()

        
    

In [ ]:
# fig, axs = plt.subplots(4*2, 2, figsize = (12, 40))
from scipy.stats import linregress
for idx, ((seqnum, hand, subnum), distances) in enumerate(distances_dict_reduced.items()):
    # if idx != 0:
    #     continue

    if subnum == 8:
        continue
    
    sub_data = subjs[(subjs['seqType'] == seqnum) & (subjs['board'] == hand) & (subjs['SubNum'] == subnum)].copy().sort_values(by = ['N']).reset_index()

    # Plot ET line plot below the heatmap for correct trials
    error_trials = all_error_trials[seqnum, hand, subnum]
    correct_trials = np.setdiff1d(np.arange(num_trials), error_trials - 1)
    correct_data = sub_data.iloc[correct_trials]


    # Plot delta Z line plot below the heatmap for correct trials
    filtered_states = reduced_forces_all_filtered_diag[seqnum, hand, subnum]
    filtered_states_correct = filtered_states[correct_trials]
    deltaz = np.diff(filtered_states_correct, axis = 0)
    deltaz = np.linalg.norm(deltaz, axis = 1)


    # Plot innovation line plot below the heatmap for correct trials
    forces = reduced_forces_all_dict[seqnum, hand, subnum]
    forces_correct = forces[correct_trials]
    innovation = np.linalg.norm(forces_correct - filtered_states_correct, axis = 1)



    # Smooth the ET values using a rolling median
    smoothed_ET = correct_data['ET'][1:]
    window_size = 5  # Adjust as needed
    smoothed_ET = smoothed_ET.rolling(window=window_size, center=True).median()

    # Remove NaN values resulting from the rolling operation
    valid_indices = smoothed_ET.notna()
    smoothed_ET = smoothed_ET[valid_indices]
    deltaz = deltaz[valid_indices]  # Align innovation data


    # Perform linear regression
    slope, intercept, r_value, p_value, std_err = linregress(smoothed_ET, deltaz)

    # Compute the best-fit line
    best_fit_line = slope * smoothed_ET + intercept

    # Compute coefficient of determination (R^2)
    r_squared = r_value**2

    print(f"Seq {seqnum} Hand {hand} Sub {subnum}", np.corrcoef(smoothed_ET, deltaz)[0, 1])

    # Plot scatter plot and best-fit line
    fig = plt.figure(figsize=(6, 6))
    plt.scatter(smoothed_ET, deltaz, s=0.5, label='Data points', alpha=0.7)
    plt.plot(smoothed_ET, best_fit_line, color='red', label=f'Best-fit line\n$R^2$={r_squared:.2f}, p={p_value:.2e}')
    plt.xlim(left = 0)
    plt.ylim(bottom = 0)


    plt.title(f"Seq {seqnum} Hand {hand} Sub {subnum}")

    plt.xlabel('ET (median smoothed)')
    plt.ylabel('deltaz')
    plt.legend()

        
    

In [ ]:
# fig, axs = plt.subplots(4*2, 2, figsize = (12, 40))
from scipy.stats import linregress
for idx, ((seqnum, hand, subnum), distances) in enumerate(distances_dict_reduced.items()):
    # if idx != 0:
    #     continue

    if subnum == 8:
        continue
    
    sub_data = subjs[(subjs['seqType'] == seqnum) & (subjs['board'] == hand) & (subjs['SubNum'] == subnum)].copy().sort_values(by = ['N']).reset_index()

    # Plot ET line plot below the heatmap for correct trials
    error_trials = all_error_trials[seqnum, hand, subnum]
    correct_trials = np.setdiff1d(np.arange(num_trials), error_trials - 1)
    correct_data = sub_data.iloc[correct_trials]


    # Plot delta Z line plot below the heatmap for correct trials
    filtered_states = reduced_forces_all_filtered_diag[seqnum, hand, subnum]
    filtered_states_correct = filtered_states[correct_trials]
    deltaz = np.diff(filtered_states_correct, axis = 0)
    deltaz = np.linalg.norm(deltaz, axis = 1)


    # Plot innovation line plot below the heatmap for correct trials
    forces = reduced_forces_all_dict[seqnum, hand, subnum]
    forces_correct = forces[correct_trials]
    innovation = np.linalg.norm(forces_correct - filtered_states_correct, axis = 1)



    # Smooth the ET values using a rolling median
    ET = correct_data['ET']
    window_size = 5  # Adjust as needed
    smoothed_ET = ET.rolling(window=window_size, center=True).median()

    # Remove NaN values resulting from the rolling operation
    valid_indices = smoothed_ET.notna()
    smoothed_ET = smoothed_ET[valid_indices]
    ET = ET[valid_indices]
    ET_high_freq = (ET - smoothed_ET) ** 2
    innovation = innovation[valid_indices]  # Align innovation data



    ET_high_freq = np.log(ET_high_freq + 1)
    # Perform linear regression
    slope, intercept, r_value, p_value, std_err = linregress(ET_high_freq, innovation)

    # Compute the best-fit line
    best_fit_line = slope * ET_high_freq + intercept

    # Compute coefficient of determination (R^2)
    r_squared = r_value**2

    print(f"Seq {seqnum} Hand {hand} Sub {subnum}", np.corrcoef(ET_high_freq, innovation)[0, 1])

    # Plot scatter plot and best-fit line
    fig = plt.figure(figsize=(6, 6))
    plt.scatter(ET_high_freq, innovation, s=0.5, label='Data points', alpha=0.7)
    plt.plot(ET_high_freq, best_fit_line, color='red', label=f'Best-fit line\n$R^2$={r_squared:.2f}, p={p_value:.2e}')
    plt.xlim(left = 0)
    plt.ylim(bottom = 0)


    plt.title(f"Seq {seqnum} Hand {hand} Sub {subnum}")

    plt.xlabel('ET (high freq)')
    plt.ylabel('innovation')
    plt.legend()

        
    

In [ ]:
#project and plot random walk on the first 3 principal components interactively in 3D
import plotly.graph_objects as go
from pykalman import KalmanFilter
import numpy.ma as ma


reduced_forces_all_smoothed_diag = {}

for idx, ((seqnum, hand, subnum), forces) in enumerate(reduced_forces_all_dict.items()):
    if subnum != 8:
        kf = custom_KM.KalmanFilter(n_dim_state = n_dim_reduction, n_dim_obs = n_dim_reduction, transition_matrices=np.eye(n_dim_reduction), observation_matrices = np.eye(n_dim_reduction), 
                    transition_offsets = np.zeros(n_dim_reduction), observation_offsets = np.zeros(n_dim_reduction), initial_state_mean=forces[0], initial_state_covariance=np.zeros((n_dim_reduction, n_dim_reduction)),
                    em_vars=['transition_covariance', 'observation_covariance'])
        

        error_inds = all_error_trials[(seqnum, hand, subnum)]
        mask = np.zeros_like(forces)
        mask[error_inds-1, :] = 1
        forces = ma.masked_array(forces, mask)
        kf = kf.em(forces, n_iter=5)
        smoothed_states, _ = kf.smooth(forces)
        reduced_forces_all_smoothed_diag[seqnum, hand, subnum] = smoothed_states

        #change color of lines continuously with index
        # color = np.arange(len(forces))
        # fig = go.Figure(data=[go.Scatter3d(x=filtered_states[:,0], y=filtered_states[:,1], z=filtered_states[:,2], mode='lines', line=dict(color=color, colorscale='Viridis', width=1, showscale = True))])
        #show color progression in colorbar
        # fig.update_layout(coloraxis_colorbar=dict(title='Index', tickvals=[0, len(forces)], ticktext=['Start', 'End']))

        # fig = go.Figure(data=[go.Scatter3d(x=forces[:,0], y=forces[:,1], z=forces[:,2], mode='lines')])
        # fig.update_layout(scene = dict(
        #                     xaxis_title='PC1',
        #                     yaxis_title='PC2',
        #                     zaxis_title='PC3'), 
        #                     width=800,
        #                     height=800, 
        #                     margin=dict(l=100, r=100, b = 100, t=100),
        #                     title_text = f'Seq {seqnum} hand {hand} sub {subnum}')
                            
        # fig.show()
        print(kf.loglikelihood(forces))




In [ ]:
for idx, ((seqnum, hand, subnum), distances) in enumerate(distances_dict_reduced.items()):
    # if idx != 0:
    #     continue

    if subnum == 8:
        continue
    
    sub_data = subjs[(subjs['seqType'] == seqnum) & (subjs['board'] == hand) & (subjs['SubNum'] == subnum)].copy().sort_values(by = ['N']).reset_index()

    

    next_BN = sub_data['BN'].shift(-1)
    sub_data['next_BN'] = next_BN
    sub_data['next_N_in_next_BN'] = (sub_data['BN'] + 1 == sub_data['next_BN'])

    block_change_trials_idx = sub_data[sub_data['next_N_in_next_BN']].index
    block_change_trials_on_plot = [np.argmin(np.abs(distances.index - trial)) for trial in block_change_trials_idx]

    first_day_change_trial_on_plot = np.argmin(np.abs(distances.index - 399))
    second_day_change_trial_on_plot = np.argmin(np.abs(distances.index - 799))





    # Start plotting
    fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(10, 12), sharex=True, gridspec_kw={'height_ratios': [4, 1, 1, 1]})



    # Plot heatmap
    sns.heatmap(distances, cmap='coolwarm', ax=ax1, cbar=False)
    ax1.axvline(first_day_change_trial_on_plot, color='black', linestyle='--')
    ax1.axvline(second_day_change_trial_on_plot, color='black', linestyle='--')
    ax1.axhline(first_day_change_trial_on_plot, color='black', linestyle='--')
    ax1.axhline(second_day_change_trial_on_plot, color='black', linestyle='--')

    # Set x-ticks and y-ticks with block change trials for the heatmap
    ax1.set_xticks(block_change_trials_on_plot)
    ax1.set_xticklabels(block_change_trials_idx)
    ax1.set_yticks(block_change_trials_on_plot)
    ax1.set_yticklabels(block_change_trials_idx)

    # Plot ET line plot below the heatmap for correct trials
    error_trials = all_error_trials[seqnum, hand, subnum]
    correct_trials = np.setdiff1d(np.arange(num_trials), error_trials - 1)
    correct_data = sub_data.iloc[correct_trials]
    ET_smoothed = correct_data['ET'].rolling(window=5, center=True).median()
    ax2.scatter(np.arange(len(ET_smoothed)) , ET_smoothed, label='ET', s = 0.5)
    ax2.axvline(first_day_change_trial_on_plot, color='black', linestyle='--')
    ax2.axvline(second_day_change_trial_on_plot, color='black', linestyle='--')

    # Align x-ticks for both subplots
    ax2.set_xticks(block_change_trials_on_plot)
    ax2.set_xticklabels(block_change_trials_idx, rotation=45)
    ax2.set_ylabel('ET')

    #despine ax2
    sns.despine(ax=ax2, left=True, bottom=True)


    # Plot delta Z line plot below the heatmap for correct trials
    smoothed_states = reduced_forces_all_filtered_diag[seqnum, hand, subnum]
    smoothed_states_correct = smoothed_states[correct_trials]
    deltaz = np.diff(smoothed_states_correct, axis = 0)
    deltaz = np.linalg.norm(deltaz, axis = 1)
    ax3.scatter(np.arange(1, len(deltaz) + 1), deltaz, s = 0.5)
    ax3.axvline(first_day_change_trial_on_plot, color='black', linestyle='--')
    ax3.axvline(second_day_change_trial_on_plot, color='black', linestyle='--')
    ax3.set_ylabel('Delta Z')
    sns.despine(ax=ax3, left=True, bottom=True)



    # Plot innovation line plot below the heatmap for correct trials
    forces = reduced_forces_all_dict[seqnum, hand, subnum]
    forces_correct = forces[correct_trials]
    innovation = np.linalg.norm(forces_correct - smoothed_states_correct, axis = 1)
    ax4.scatter(np.arange(0, len(innovation)), innovation, s = 0.5)
    ax4.axvline(first_day_change_trial_on_plot, color='black', linestyle='--')
    ax4.axvline(second_day_change_trial_on_plot, color='black', linestyle='--')
    ax4.set_ylabel('Innovation')
    sns.despine(ax=ax4, left=True, bottom=True)




    #setting title for whole figure
    fig.suptitle(f"Seq {seqnum} Hand {hand} Sub {subnum}")
    

    # Add labels and titles as needed
    # ax1.set_title("Heatmap with Day Change Lines")
    # ax2.set_title("ET over Trials")

    plt.tight_layout(rect=[0, 0, 1, 0.97])
    plt.subplots_adjust(hspace=0.05)  # Reduce space between subplots for alignment
    plt.show()

        
    